In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tools import *
from reload import *
from models.convNet_v1 import *

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
dataset = reload()

[ Load Dataset ]: 100%|██████████| 5067/5067 [00:15<00:00, 330.34it/s]


In [3]:
dataset.train_dataset = MinMaxScaler(dataset.train_dataset.reshape(3375,-1)).reshape(3375,64,64,64)
dataset.test_dataset = MinMaxScaler(dataset.test_dataset.reshape(1692,-1)).reshape(1692,64,64,64)

In [4]:
tf.reset_default_graph()
convnet = convNet((64,64,64), (5,))
score = convnet.fc8

train_layers = ['fc8', 'fc7', 'fc6']
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers]

with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score,labels=convnet.y))

In [5]:
with tf.name_scope("train"):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.001,rho=0.95,epsilon=1e-09).minimize(loss)
    #optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
for var in var_list:
    tf.summary.histogram(var.name, var)
    
tf.summary.scalar('cross_entropy', loss)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(convnet.y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
tf.summary.scalar('accuracy', accuracy)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./out")

saver = tf.train.Saver()

INFO:tensorflow:Summary name fc6/weights:0 is illegal; using fc6/weights_0 instead.
INFO:tensorflow:Summary name fc6/biases:0 is illegal; using fc6/biases_0 instead.
INFO:tensorflow:Summary name fc7/weights:0 is illegal; using fc7/weights_0 instead.
INFO:tensorflow:Summary name fc7/biases:0 is illegal; using fc7/biases_0 instead.
INFO:tensorflow:Summary name fc8/weights:0 is illegal; using fc8/weights_0 instead.
INFO:tensorflow:Summary name fc8/biases:0 is illegal; using fc8/biases_0 instead.


In [ ]:
train_dataset, train_labels, test_dataset, test_labels, tr_batch_num, te_batch_num = make_batchs(100,
                                                                    dataset.train_dataset, dataset.train_labels,
                                                                    dataset.test_dataset, dataset.test_labels)

In [ ]:
num_epochs=10000

with tf.device('/gpu:0'):
    convnet.sess.run(tf.global_variables_initializer())
    
    with tqdm(total = num_epochs) as pbar:
        for epoch in range(num_epochs):
            train_acc = 0.
            train_loss = 0.
            train_count = 0
            for i in range(tr_batch_num):
                acc,cost,op = convnet.sess.run([accuracy,loss,optimizer], feed_dict={convnet.x: train_dataset[i],
                                                                             convnet.y: one_hot_encoding(train_labels[i]-np.array([1])),
                                                                             convnet.keep_prob: 0.7})
                train_acc += acc
                train_loss += cost
                train_count += 1
                
            test_acc = 0.
            test_loss = 0.
            test_count = 0
            for i in range(te_batch_num):
                acc,cost = convnet.sess.run([accuracy,loss], feed_dict={convnet.x: test_dataset[i],
                                                                    convnet.y: one_hot_encoding(test_labels[i]-np.array([1])),
                                                                    convnet.keep_prob: 1.0})
                test_acc += acc
                test_loss += cost
                test_count += 1

            pbar.set_postfix_str(" Test - Accuracy : {:.5f}".format(test_acc/test_count if test_count !=0 else 0)+
                                     " Test - Loss : {:.5f}".format(test_loss/test_count if test_count !=0 else 0))
                    
            pbar.set_description("[Epochs : "+str(epoch+1)+" ]"+
                                 " Train - Accuracy : {:.5f}".format(train_acc/train_count if train_count !=0 else 0)+
                                 " Train - Loss : {:.5f}".format(train_loss/train_count if train_count !=0 else 0))
            pbar.update(1)

[Epochs : 1962 ] Train - Accuracy : 0.99824 Train - Loss : 0.03551:  20%|█▉        | 1962/10000 [6:51:50<27:07:36, 12.15s/it,  Test - Accuracy : 0.52731 Test - Loss : 1.58337]